In [2]:
# File Name: SageCheckIWnkViaILPAgainstSignedPermutationsDate29082022Ver01.txt
# Purpose: Check (Integer) Weighing matrices Via Integer Linear Programming against (direct) Signed permutations
# Remark: After session with Radi. We are left with a question:
#         Why No solution for R1,R2:  [0, 0, 0, 0, 0, 0, 5] [0, 0, 2, 2, 2, 2, 3]
# Improvements: (1) Problem from remark fixed.
#               (2) Added "sors" routine from Assaf, so to generalize the whole activity.
# Remark: Radi on 8/5/22 noticed that solutions begin with negative values although BegWithPos should prevent it.
#         We fix it.
# Remark: On 12/5/22 we added two routines
#         OurMain2: Given parameters of weighing matrix, weight and 2 divisors of order, it gives all pairs of
#                   orthogonal rows of all types available of nsoks and filter them by the code invariany method.
#         AddLine:  Given Nsoks and Sols gives all additions to Sols by one row taken from Nsoks
#                   with permutations and signings and codeinvariant.
#                   This routine is the basis for recursion to construct all Integer Weighing Matrices.
#                   We have not run and debugged this routine yet. Next time...
# Remark: On 15/5/22 we wroteqimproved/touched some routines:
#         OurMain2: This is the new main. It gets the [WMweight, WMOrderDiv] of a weighing matrix and should output all inequivalent Integer weighing matrices of order=WMOrderDiv & weight=sqrt(WMweight. 
#         FilterOrthogonalSignedPermutationsAgainstMatrix (haven't debugged yet)
#         CodePrevRows (debugged carefully)
# Remark: On 26052022 we debuged OurMain2 routine.
# Remark On 30-6-22 Organizing the hp check. 
#                   automatic base calculation for the code invariant
#                   indexing the permutation and signs and saving the index in the configuration.
# Remark on 3-7-22 Adding code to check for Hadamrd equivalnce for two rows for permutation AND row negation
#                  Changes in "FilterOrthogonalSignedPermutationsByCodeInvariant" adding two code invariants for each line.
# Remark on 14-7-22 Adding the index for each line of (NsoksIndex,PermutationIndex,SignsIndex) 
#                    Bug for line [3, 2, 2, 2, 2, 0, 0], [[1, 16, 5], (2, -3, 2, -2, 0, 2, 0)]], 
#                                 The SignsIndex looks incorrect - TBD in the next meeting
# Remark on 21-7-22 Generelizing for AboveMatrix.
#                    -   OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                    -   normAboveAgainstMatrix(AboveMatrix,R2):
#                    -   dpAgainstMatrix
#                       need to correct FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
# Remark on 24-7-22  began adapting FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                    adding routine genCode for generating a code for the column i.
#                   for next time - continue to adapt  FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                                    and remove CodePrevRows
# Remark on 31-7-22 remove CodePrevTows and AddLine
#                   Checked FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix for addition of one row.
#                   Coded OurMain3 to work with FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                   Checked OurMain3 for two lines.
# Remark on 31-7-22 Adapt OurMain3 for 7 lines
#                   While running OurMain3 it works fine for 2 and 3 lines but crashes while
#                             calculating for 4 lines in memory shortage
# Remark on 18-7-22 Cleanup
# Remark on 28-8-22 
#                  integrate   FilterOrthogonalSignedPermutationsByCodeInvariant
#                           to OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                  integrate hp into OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                  integrate gencode also
#                  add goodPerm test to eliminate (-1) assignment to 0 (faster)


def HadamardSpace(d):
    SpaceSize = 2^d
    L=[ZZ(n).digits(2) for n in range(SpaceSize)]
    L1 = [x+[0]*(d-len(x)) for x in L]
    return([vector([2*y-1 for y in x]) for x in L1])

def IsBegWithPos(L):
    for x in L:
        if x>0:
            return(True)
        if x<0:
            return(False)
    return(False)

def goodPerm(L,s):
    for i in range(len(L)):
        if (L[i]==0) and (s[i]==-1) : return false
    return true

def normAboveAgainstMatrix(AboveMatrix,R2):
    for i in range(len(R2)):
        if AboveMatrix.column(i).norm()==0:
            R2[i]=abs(R2[i])
    return R2


def dpAgainstMatrix(AboveMatrix,L2):
    return((AboveMatrix*vector(L2)).norm())


def OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix(AboveMat,L,Lindex,MaxV):
    P = Permutations(L).list()
    d = len(L)
    S = HadamardSpace(d)
    SP = []
    s = []
    sd = [] 
    codeBase = 1+2*MaxV
    nR=AboveMat.nrows()
    cB=(vector([codeBase^(nR-j) for j in range(nR)]))*AboveMat
    for pi in range(Lindex[1],len(P)):
        p = P[pi]
        s0=Lindex[2] if pi==Lindex[1] else 0
        for si in range(s0,len(S)):
            sn=S[si]
            if (goodPerm(p,sn)):
                r =[p[i]*sn[i] for i in range(d)]
                if IsBegWithPos(r):
                    if dpAgainstMatrix(AboveMat,r)==0:
                        r=normAboveAgainstMatrix(AboveMat,r)
                        if not(r in SP):
                            SP.append(r)
                            R2i=vector(r)
                            M1 = cB+R2i
                            M1s = sorted(M1)
                            if not(M1s in s) :
                                M1m = cB-R2i
                                M1sm = sorted(M1m)
                                if not(M1sm in s):
                                    s.append(M1s)
                                    sd.append([[Lindex[0],pi,si],R2i])                    
    return(sd)



def sors(n,r,maxsq):  ### Find all ways to express n=\sum_i s_i^2 as a sum of r squares, with maxsq>=s_1>=s_2>=...>=s_r.
    #n0=RDF(n)
    n0=n
    if r==1:
        s=sqrt(n0)
        fs=floor(s)
        if s==fs and s<=maxsq:
            return [[s]]
        else:
            return false
    else:
        sors2=[]
        for x in range(floor(sqrt(n0/r)),min(floor(sqrt(n0)),maxsq)+1):
            m=n-x^2
            #print x
            sors1=sors(m,r-1,x)
            if sors1:
                sors2+=[[x]+sors1[i] for i in range(len(sors1))   ]
           # print "sors2=",sors2
        if len(sors2)>0:
            return sors2
        else:
            return false


def OurMain3(WMWeight, WMOrderDiv,lines):
    MaxSquare = WMWeight^(1/2)
    AllSoks = sors(WMWeight, WMOrderDiv, MaxSquare)
    print("Nsoks: ",AllSoks)
    Sols= [[matrix(1,WMOrderDiv,AllSoks[i]),[i,0,0]] for i in range(len(AllSoks))]
    print(Sols)
    for line in range(lines-1):
        nSols=[]
        countSol=0
        countScan=0
        for Sol in Sols:
            countScan+=1
            AboveMat=Sol[0]
            R1Index=Sol[1]
            for R2i in range(R1Index[0],len(AllSoks)):
                R2Index=R1Index if R2i==R1Index[0] else [R2i,0,0]
                R2 = vector(AllSoks[R2i])
                cpF = OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix(AboveMat,R2,R2Index,MaxSquare)
                countSol+=len(cpF)
                if len(cpF)>0:
                    nSols.extend([AboveMat,x] for x in cpF)
            print("countScan,countSol  = ",countScan,countSol)
        Sols=[]
        for nSol in nSols:
            NL=matrix(WMOrderDiv,1,list(nSol[1][1]))
            AM=((nSol[0].transpose()).augment(NL)).transpose()
            Sols.append([AM,nSol[1][0]])
        print("# of sols in line",line, "  =",len(Sols),[Sols[i] for i in range(min(len(Sols),3))])
    return(Sols)

from gc import collect
collect()
#%time res=OurMain3(8,8,8)

7

In [89]:
save (res,'IW(8,8)')

In [90]:
res = load('IW(8,8)')

In [92]:
res

[[
[ 1  1  1  1  1  1  1  1]             
[ 1  1  1  1 -1 -1 -1 -1]             
[ 1  1 -1 -1  1  1 -1 -1]             
[ 1 -1  1 -1  1 -1  1 -1]             
[ 1 -1 -1  1 -1  1  1 -1]             
[ 1 -1 -1  1  1 -1 -1  1]             
[ 1 -1  1 -1 -1  1 -1  1]             
[ 1  1 -1 -1 -1 -1  1  1], [0, 0, 195]
],
 [
[ 1  1  1  1  1  1  1  1]               
[ 1  1  1  1 -1 -1 -1 -1]               
[ 1  1 -1 -1  1  1 -1 -1]               
[ 1 -1  1 -1  1 -1  1 -1]               
[ 2 -1 -1  0 -1  0  0  1]               
[ 0  1  1 -2 -1  0  0  1]               
[ 0  1 -1  0  1 -2  0  1]               
[ 0  1 -1  0 -1  0  2 -1], [1, 225, 107]
],
 [
[ 1  1  1  1  1  1  1  1]               
[ 1  1  1  1 -1 -1 -1 -1]               
[ 1  1 -1 -1  1  1 -1 -1]               
[ 1 -1  1 -1  1 -1  1 -1]               
[ 1 -2  0  1  0  1 -1  0]               
[ 1  0 -2  1  0 -1  1  0]               
[ 1  0  0 -1 -2  1  1  0]               
[ 1  0  0 -1  0 -1 -1  2], [1, 170, 151]
],
 [
[ 1  1  1  

In [126]:
L=8*[1,-1]

In [127]:
p=Permutations(L).list()
len(p)

12870

In [3]:
%time res=OurMain3(25,7,7)

Nsoks:  [[2, 2, 2, 2, 2, 2, 1], [3, 2, 2, 2, 2, 0, 0], [3, 3, 2, 1, 1, 1, 0], [4, 2, 1, 1, 1, 1, 1], [4, 2, 2, 1, 0, 0, 0], [4, 3, 0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0, 0]]
[[[2 2 2 2 2 2 1], [0, 0, 0]], [[3 2 2 2 2 0 0], [1, 0, 0]], [[3 3 2 1 1 1 0], [2, 0, 0]], [[4 2 1 1 1 1 1], [3, 0, 0]], [[4 2 2 1 0 0 0], [4, 0, 0]], [[4 3 0 0 0 0 0], [5, 0, 0]], [[5 0 0 0 0 0 0], [6, 0, 0]]]
countScan,countSol  =  1 6
countScan,countSol  =  2 23
countScan,countSol  =  3 95
countScan,countSol  =  4 103
countScan,countSol  =  5 119
countScan,countSol  =  6 122
countScan,countSol  =  7 123
# of sols in line 0   = 123 [[[ 2  2  2  2  2  2  1]
[ 2  2  2 -2 -2 -1 -2], [0, 1, 7]], [[ 2  2  2  2  2  2  1]
[ 3  2 -2 -2  0  0 -2], [1, 2, 51]], [[ 2  2  2  2  2  2  1]
[ 3 -3  1  1 -1  0 -2], [2, 10, 45]]]
countScan,countSol  =  1 4
countScan,countSol  =  2 11
countScan,countSol  =  3 27
countScan,countSol  =  4 29
countScan,countSol  =  5 31
countScan,countSol  =  6 34
countScan,countSol  =  7 42
countScan,cou

countScan,countSol  =  117 225
countScan,countSol  =  118 227
countScan,countSol  =  119 228
countScan,countSol  =  120 229
countScan,countSol  =  121 230
countScan,countSol  =  122 232
countScan,countSol  =  123 236
countScan,countSol  =  124 239
countScan,countSol  =  125 241
countScan,countSol  =  126 244
countScan,countSol  =  127 248
countScan,countSol  =  128 249
countScan,countSol  =  129 250
countScan,countSol  =  130 250
countScan,countSol  =  131 250
countScan,countSol  =  132 253
countScan,countSol  =  133 255
countScan,countSol  =  134 256
countScan,countSol  =  135 257
countScan,countSol  =  136 257
countScan,countSol  =  137 258
countScan,countSol  =  138 258
countScan,countSol  =  139 258
countScan,countSol  =  140 258
countScan,countSol  =  141 258
countScan,countSol  =  142 258
countScan,countSol  =  143 258
countScan,countSol  =  144 258
countScan,countSol  =  145 258
countScan,countSol  =  146 258
countScan,countSol  =  147 258
countScan,countSol  =  148 258
countSca

countScan,countSol  =  382 775
countScan,countSol  =  383 779
countScan,countSol  =  384 787
countScan,countSol  =  385 790
countScan,countSol  =  386 793
countScan,countSol  =  387 796
countScan,countSol  =  388 799
countScan,countSol  =  389 802
countScan,countSol  =  390 804
countScan,countSol  =  391 806
countScan,countSol  =  392 807
countScan,countSol  =  393 808
countScan,countSol  =  394 810
countScan,countSol  =  395 812
countScan,countSol  =  396 813
countScan,countSol  =  397 813
countScan,countSol  =  398 814
countScan,countSol  =  399 814
countScan,countSol  =  400 814
countScan,countSol  =  401 818
countScan,countSol  =  402 822
countScan,countSol  =  403 826
countScan,countSol  =  404 829
countScan,countSol  =  405 831
countScan,countSol  =  406 834
countScan,countSol  =  407 837
countScan,countSol  =  408 840
countScan,countSol  =  409 842
countScan,countSol  =  410 845
countScan,countSol  =  411 848
countScan,countSol  =  412 853
countScan,countSol  =  413 855
countSca

countScan,countSol  =  642 1220
countScan,countSol  =  643 1221
countScan,countSol  =  644 1222
countScan,countSol  =  645 1223
countScan,countSol  =  646 1223
countScan,countSol  =  647 1223
countScan,countSol  =  648 1223
countScan,countSol  =  649 1223
countScan,countSol  =  650 1223
countScan,countSol  =  651 1223
countScan,countSol  =  652 1225
countScan,countSol  =  653 1226
countScan,countSol  =  654 1229
countScan,countSol  =  655 1232
countScan,countSol  =  656 1235
countScan,countSol  =  657 1236
countScan,countSol  =  658 1238
countScan,countSol  =  659 1239
countScan,countSol  =  660 1240
countScan,countSol  =  661 1241
countScan,countSol  =  662 1241
countScan,countSol  =  663 1241
countScan,countSol  =  664 1241
countScan,countSol  =  665 1241
countScan,countSol  =  666 1241
countScan,countSol  =  667 1241
countScan,countSol  =  668 1242
countScan,countSol  =  669 1243
countScan,countSol  =  670 1244
countScan,countSol  =  671 1246
countScan,countSol  =  672 1247
countSca

countScan,countSol  =  899 1474
countScan,countSol  =  900 1477
countScan,countSol  =  901 1480
countScan,countSol  =  902 1482
countScan,countSol  =  903 1484
countScan,countSol  =  904 1486
countScan,countSol  =  905 1487
countScan,countSol  =  906 1488
countScan,countSol  =  907 1489
countScan,countSol  =  908 1491
countScan,countSol  =  909 1493
countScan,countSol  =  910 1494
countScan,countSol  =  911 1495
countScan,countSol  =  912 1495
countScan,countSol  =  913 1498
countScan,countSol  =  914 1500
countScan,countSol  =  915 1501
countScan,countSol  =  916 1503
countScan,countSol  =  917 1504
countScan,countSol  =  918 1505
countScan,countSol  =  919 1507
countScan,countSol  =  920 1509
countScan,countSol  =  921 1510
countScan,countSol  =  922 1511
countScan,countSol  =  923 1512
countScan,countSol  =  924 1514
countScan,countSol  =  925 1515
countScan,countSol  =  926 1517
countScan,countSol  =  927 1519
countScan,countSol  =  928 1520
countScan,countSol  =  929 1521
countSca

...

countScan,countSol  =  450 235
countScan,countSol  =  451 235
countScan,countSol  =  452 236
countScan,countSol  =  453 236
countScan,countSol  =  454 238
countScan,countSol  =  455 239
countScan,countSol  =  456 239
countScan,countSol  =  457 240
countScan,countSol  =  458 240
countScan,countSol  =  459 240
countScan,countSol  =  460 240
countScan,countSol  =  461 241
countScan,countSol  =  462 241
countScan,countSol  =  463 242
countScan,countSol  =  464 242
countScan,countSol  =  465 242
countScan,countSol  =  466 243
countScan,countSol  =  467 243
countScan,countSol  =  468 244
countScan,countSol  =  469 244
countScan,countSol  =  470 244
countScan,countSol  =  471 244
countScan,countSol  =  472 245
countScan,countSol  =  473 245
countScan,countSol  =  474 247
countScan,countSol  =  475 248
countScan,countSol  =  476 248
countScan,countSol  =  477 248
countScan,countSol  =  478 250
countScan,countSol  =  479 251
countScan,countSol  =  480 251
countScan,countSol  =  481 252
countSca

countScan,countSol  =  715 356
countScan,countSol  =  716 357
countScan,countSol  =  717 357
countScan,countSol  =  718 357
countScan,countSol  =  719 358
countScan,countSol  =  720 358
countScan,countSol  =  721 358
countScan,countSol  =  722 358
countScan,countSol  =  723 358
countScan,countSol  =  724 360
countScan,countSol  =  725 361
countScan,countSol  =  726 361
countScan,countSol  =  727 361
countScan,countSol  =  728 361
countScan,countSol  =  729 361
countScan,countSol  =  730 361
countScan,countSol  =  731 361
countScan,countSol  =  732 363
countScan,countSol  =  733 364
countScan,countSol  =  734 364
countScan,countSol  =  735 365
countScan,countSol  =  736 365
countScan,countSol  =  737 365
countScan,countSol  =  738 367
countScan,countSol  =  739 368
countScan,countSol  =  740 368
countScan,countSol  =  741 369
countScan,countSol  =  742 369
countScan,countSol  =  743 369
countScan,countSol  =  744 371
countScan,countSol  =  745 372
countScan,countSol  =  746 372
countSca

countScan,countSol  =  980 451
countScan,countSol  =  981 451
countScan,countSol  =  982 451
countScan,countSol  =  983 451
countScan,countSol  =  984 451
countScan,countSol  =  985 451
countScan,countSol  =  986 451
countScan,countSol  =  987 452
countScan,countSol  =  988 452
countScan,countSol  =  989 452
countScan,countSol  =  990 452
countScan,countSol  =  991 452
countScan,countSol  =  992 453
countScan,countSol  =  993 453
countScan,countSol  =  994 453
countScan,countSol  =  995 454
countScan,countSol  =  996 454
countScan,countSol  =  997 454
countScan,countSol  =  998 454
countScan,countSol  =  999 454
countScan,countSol  =  1000 455
countScan,countSol  =  1001 455
countScan,countSol  =  1002 455
countScan,countSol  =  1003 455
countScan,countSol  =  1004 455
countScan,countSol  =  1005 456
countScan,countSol  =  1006 456
countScan,countSol  =  1007 456
countScan,countSol  =  1008 457
countScan,countSol  =  1009 457
countScan,countSol  =  1010 457
countScan,countSol  =  1011 

countScan,countSol  =  1 1
countScan,countSol  =  2 1
countScan,countSol  =  3 1
countScan,countSol  =  4 1
countScan,countSol  =  5 2
countScan,countSol  =  6 2
countScan,countSol  =  7 3
countScan,countSol  =  8 3
countScan,countSol  =  9 3
countScan,countSol  =  10 3
countScan,countSol  =  11 3
countScan,countSol  =  12 3
countScan,countSol  =  13 4
countScan,countSol  =  14 4
countScan,countSol  =  15 4
countScan,countSol  =  16 4
countScan,countSol  =  17 5
countScan,countSol  =  18 5
countScan,countSol  =  19 5
countScan,countSol  =  20 5
countScan,countSol  =  21 6
countScan,countSol  =  22 6
countScan,countSol  =  23 6
countScan,countSol  =  24 7
countScan,countSol  =  25 7
countScan,countSol  =  26 7
countScan,countSol  =  27 7
countScan,countSol  =  28 7
countScan,countSol  =  29 7
countScan,countSol  =  30 7
countScan,countSol  =  31 7
countScan,countSol  =  32 8
countScan,countSol  =  33 8
countScan,countSol  =  34 9
countScan,countSol  =  35 10
countScan,countSol  =  36 11

countScan,countSol  =  279 63
countScan,countSol  =  280 63
countScan,countSol  =  281 63
countScan,countSol  =  282 63
countScan,countSol  =  283 63
countScan,countSol  =  284 64
countScan,countSol  =  285 64
countScan,countSol  =  286 64
countScan,countSol  =  287 64
countScan,countSol  =  288 64
countScan,countSol  =  289 65
countScan,countSol  =  290 65
countScan,countSol  =  291 65
countScan,countSol  =  292 65
countScan,countSol  =  293 65
countScan,countSol  =  294 65
countScan,countSol  =  295 65
countScan,countSol  =  296 65
countScan,countSol  =  297 65
countScan,countSol  =  298 65
countScan,countSol  =  299 66
countScan,countSol  =  300 66
countScan,countSol  =  301 66
countScan,countSol  =  302 67
countScan,countSol  =  303 67
countScan,countSol  =  304 67
countScan,countSol  =  305 67
countScan,countSol  =  306 67
countScan,countSol  =  307 68
countScan,countSol  =  308 68
countScan,countSol  =  309 68
countScan,countSol  =  310 68
countScan,countSol  =  311 68
countScan,

In [4]:
res

[[
[ 2  2  2  2  0  0]            
[ 2  2 -2 -2  0  0]            
[ 2 -2  2 -2  0  0]            
[ 2 -2 -2  2  0  0]            
[ 0  0  0  0  4  0]            
[ 0  0  0  0  0  4], [2, 5, 63]
],
 [
[ 2  2  2  2  0  0]             
[ 2  2 -2 -2  0  0]             
[ 2 -2  0  0  2  2]             
[ 2 -2  0  0 -2 -2]             
[ 0  0  2 -2  2 -2]             
[ 0  0  2 -2 -2  2], [0, 14, 39]
],
 [
[ 2  2  2  2  0  0]              
[ 2 -2  0  0  2  2]              
[ 2  0 -3  1 -1 -1]              
[ 2  0  1 -3 -1 -1]              
[ 0  2 -1 -1  3 -1]              
[ 0  2 -1 -1 -1  3], [1, 107, 35]
],
 [
[ 2  2  2  2  0  0]              
[ 2 -2  0  0  2  2]              
[ 2  0 -1 -1 -3  1]              
[ 2  0 -1 -1  1 -3]              
[ 0  2 -3  1  1  1]              
[ 0  2  1 -3  1  1], [1, 105, 55]
],
 [
[ 2  2  2  2  0  0]            
[ 3 -1 -1 -1  2  0]            
[ 1 -3  1  1 -2  0]            
[ 1  1 -3  1 -2  0]            
[ 1  1  1 -3 -2  0]            
[ 0  0  0  0  0

In [4]:
OurMain3(16,6,6)

Nsoks:  [[2, 2, 2, 2, 0, 0], [3, 2, 1, 1, 1, 0], [4, 0, 0, 0, 0, 0]]
[[[2 2 2 2 0 0], [0, 0, 0]], [[3 2 1 1 1 0], [1, 0, 0]], [[4 0 0 0 0 0], [2, 0, 0]]]
countScan,countSol  =  1 9
countScan,countSol  =  2 20
countScan,countSol  =  3 21
# of sols in line 0   = 21 [[[ 2  2  2  2  0  0]
[ 2  2 -2 -2  0  0], [0, 0, 51]], [[ 2  2  2  2  0  0]
[ 2 -2  0  0  2  2], [0, 5, 13]], [[ 2  2  2  2  0  0]
[ 3 -2 -1  0  1  1], [1, 2, 9]]]
countScan,countSol  =  1 4
countScan,countSol  =  2 10
countScan,countSol  =  3 12
countScan,countSol  =  4 14
countScan,countSol  =  5 16
countScan,countSol  =  6 18
countScan,countSol  =  7 20
countScan,countSol  =  8 22
countScan,countSol  =  9 23
countScan,countSol  =  10 27
countScan,countSol  =  11 31
countScan,countSol  =  12 34
countScan,countSol  =  13 37
countScan,countSol  =  14 40
countScan,countSol  =  15 42
countScan,countSol  =  16 45
countScan,countSol  =  17 48
countScan,countSol  =  18 48
countScan,countSol  =  19 48
countScan,countSol  =  20 48
c

[[
[ 2  2  2  2  0  0]            
[ 2  2 -2 -2  0  0]            
[ 2 -2  2 -2  0  0]            
[ 2 -2 -2  2  0  0]            
[ 0  0  0  0  4  0]            
[ 0  0  0  0  0  4], [2, 5, 63]
],
 [
[ 2  2  2  2  0  0]             
[ 2  2 -2 -2  0  0]             
[ 2 -2  0  0  2  2]             
[ 2 -2  0  0 -2 -2]             
[ 0  0  2 -2  2 -2]             
[ 0  0  2 -2 -2  2], [0, 14, 39]
],
 [
[ 2  2  2  2  0  0]              
[ 2 -2  0  0  2  2]              
[ 2  0 -3  1 -1 -1]              
[ 2  0  1 -3 -1 -1]              
[ 0  2 -1 -1  3 -1]              
[ 0  2 -1 -1 -1  3], [1, 107, 35]
],
 [
[ 2  2  2  2  0  0]              
[ 2 -2  0  0  2  2]              
[ 2  0 -1 -1 -3  1]              
[ 2  0 -1 -1  1 -3]              
[ 0  2 -3  1  1  1]              
[ 0  2  1 -3  1  1], [1, 105, 55]
],
 [
[ 2  2  2  2  0  0]            
[ 3 -1 -1 -1  2  0]            
[ 1 -3  1  1 -2  0]            
[ 1  1 -3  1 -2  0]            
[ 1  1  1 -3 -2  0]            
[ 0  0  0  0  0

In [105]:
p=[i^2  for i in range(10)]

In [118]:
from operator import mul
r=list(map(mul,p,p))
print(r)

[0, 1, 16, 81, 256, 625, 1296, 2401, 4096, 6561]
